In [1]:
import requests, json
from tqdm import tqdm
from datetime import datetime
import pandas as pd
import urllib
import time

api = '2A955004026F4AEAAFA21FBCCA4A65FB'

In [2]:
with open("asins.txt", "r") as f:
    asins = f.read().split('\n')
asins = [x for x in asins if x != ""]
len(asins)

962

In [3]:
# # paste asins here
# asins = [x for x in """

# B07FMQ8TV4
# B015REH9L6
# B0083COYZO
# B005GRBH4K
# B000TDMUH4

# """.split('\n') if x != '' and len(x) > 4]
# print(len(asins))
# print(len(set(asins)))

In [5]:
# make collection
def make_coll():
    body = {
      "name": datetime.strftime(datetime.now(), '%Y-%m-%d %H:%M'),
      "enabled": True,
      "schedule_type": "manual"
    }
    api_result = requests.post(
        'https://api.rainforestapi.com/collections?api_key={}'.format(api), 
        json=body
    )
    api_response = api_result.json()
    col = api_response['collection']['id']
    return col

# make offers payload
def make_payload(type_, type_key, list_):
    requests_ = []
    for asin in list_:
        request = {
            'type': type_,
            'amazon_domain': 'amazon.com',
            type_key: asin,
            'custom_id': '123'
        }
        requests_.append(request)
    body = {
        'requests': requests_
    }
    return body

# add to collection
def add_to_coll(col, body):
    params = {
      'api_key': api
    }
    result = requests.put(
        'https://api.rainforestapi.com/collections/{}?api_key={}'.format(col, api), 
        json=body
    )
    r = result.json()
    return r

# start col
def start_coll(col):
    params = {
      'api_key': api
    }
    result = requests.get('https://api.rainforestapi.com/collections/{}/start'.format(col), params)
    r = result.json()
    return r

# check col
def check_coll(col):
    params = {
      'api_key': api
    }
    result = requests.get(
        'https://api.rainforestapi.com/collections/{}'.format(col), 
        params
    )
    response = result.json()
    return response

# get download link
def get_dl(col):
    params = {
      'api_key': api
    }
    api_result = requests.get(
        'https://api.rainforestapi.com/collections/{}/results/1'.format(col), 
        params
    )
    r = api_result.json()
    url = r['result']['download_links']['pages'][0]
    response = urllib.request.urlopen(url)
    data = json.loads(response.read())
    return data

def check_until_done(col):
    while True:
        r = check_coll(col)
        print(r['collection']['status'])
        if r['collection']['status'] != 'idle':
            time.sleep(10)
        else:
            break

In [69]:
def run_get_offers(asins):
    col = make_coll()
    body = make_payload('offers', 'asin', list(set(asins)))
    add_to_coll(col, body)
    start_coll(col)
    check_until_done(col)
    return col

In [6]:
# col = make_coll()

In [7]:
# body = make_payload('offers', 'asin', list(set(asins)))

In [70]:
# add_to_coll(col, body)

In [71]:
# start_coll(col)

In [10]:
# check_coll(col)

In [68]:
# check_until_done(col)

In [ ]:
col = run_get_offers(asins)

In [13]:
data = get_dl(col)

In [14]:
def build_offers_req(data):
    offers = []
    for x in data:
        asin = x['request']['asin']
        if 'offers' in x['result']:
            for y in x['result']['offers']:
                offer_name = y['seller']['name']
                offer_id = y['seller']['id'] if 'id' in y['seller'] else None
                offers.append({
                    'asin': asin,
                    'offer_name': offer_name,
                    'offer_id': offer_id
                })
        else:
            offers.append({
                    'asin': asin,
                    'offer_name': None,
                    'offer_id': None
                })
    return offers

In [15]:
offers = build_offers_req(data)

In [17]:
# offers = [{
#     'asin': x['request']['asin'],
#     'offers': [{
#         'name': y['seller']['name'],
#         'id': y['seller']['id'] if 'id' in y['seller'] else None
#     } if len(x['result']['offers']) else None for y in x['result']['offers']]
# } for x in data]

# [
#     {
#         'asin': x['request']['asin'],
#         'name': y['seller']['name'],
#         'id': y['seller']['id']
#     } if len(x['result']['offers']) else None for y in x['result']['offers'] for x in data
# ]

# [{
#         'asin': x['request']['asin'],
#         'name': y['seller']['name'],
#         'id': y['seller']['id']
#     } if len(y) else None for y in x['result']['offers'] for x in data]

In [18]:
# [x for x in pd.DataFrame(offers)['offer_id'].unique() if x != None]

In [19]:
# body = make_payload('seller_profile', 'seller_id', [x for x in pd.DataFrame(offers)['offer_id'].unique() if x != None])

In [27]:
# len(body['requests'])

In [51]:
def get_all_sellers(offers):
    body = make_payload(
        'seller_profile', 
        'seller_id', 
        [x for x in pd.DataFrame(offers)['offer_id'].unique() if x != None]
    )
    print(len(body['requests']))
    col = make_coll()
    add_to_coll(col, body)
    start_coll(col)
    check_until_done(col)
    return col

In [29]:
# col = make_coll()

In [30]:
# add_to_coll(col, body)

In [31]:
# start_coll(col)

In [32]:
# check_until_done(col)

In [33]:
# check_coll(col)

In [35]:
col = get_all_sellers(offers)

100
queued
running
running
idle


In [67]:
# params = {
#   'api_key': api
# }

# api_result = requests.get('https://api.rainforestapi.com/collections', params)
# api_response = api_result.json()

# print ("Total Collections: ", api_response['total_count'])
# for number, collection in enumerate(api_response['collections'], start=1):
#     print ("%s. %s" % (number, collection['name']))

# col = api_response['collections'][-1]['id']

In [59]:
data = get_dl(col)

In [60]:
def clean_data(data):
    result = []
    for x in data:
        result.append({
            'id': x['request']['seller_id'],
            'url': x['result']['request_metadata']['amazon_url'],
            'detail_name': x['result']['seller_details']['name'],
            'detail_address': x['result']['seller_details']['detailed_information'],
        })
    result = pd.DataFrame(result)
    result['address'] = result.apply(
        lambda x: x['detail_address'].replace('Business Name: ', '').replace('Business Address: ', ''),
        axis=1
    )
    result.drop(columns=['detail_address'], inplace=True)
    return result

In [61]:
# result = pd.DataFrame(result)

In [62]:
# result['address'] = result.apply(lambda x: x['detail_address'].replace('Business Name: ', '').replace('Business Address: ', ''), axis=1)

In [63]:
# result.drop(columns=['detail_address'], inplace=True)

In [64]:
result = clean_data(data)

In [65]:
result

,id,url,detail_name,address
0,AW729ASF30E9K,https://www.amazon.com/sp?seller=AW729ASF30E9K,me+,DIEGO L GIOL 412 N. Main Street Buffalo Wyomin...
1,AQUCGX4T1LI6Z,https://www.amazon.com/sp?seller=AQUCGX4T1LI6Z,VASU ENTERPRISE.,"Viveksheel Chhibber 9-10, Street number 13A, C..."
2,A1OVKRUZFQVFU,https://www.amazon.com/sp?seller=A1OVKRUZFQVFU,Natural Nutrient,SandJohn 970 W Valley Pkwy #417 Escondido CA 9...
3,A2JUJI5HUCMXMN,https://www.amazon.com/sp?seller=A2JUJI5HUCMXMN,IhoStore,UNIGRANDCOMMERCE LLC 7901 4th. St. N. Ste.300 ...
4,A19NVE4G6SOT2C,https://www.amazon.com/sp?seller=A19NVE4G6SOT2C,Kaspien,"etailz, Inc. 2818 N. Sullivan Suite 130 Spokan..."
...,...,...,...,...
95,A1SREZ8ZP696ZP,https://www.amazon.com/sp?seller=A1SREZ8ZP696ZP,Holly Hill Health Foods,Holly Hill Health Foods ATTN CAROLYN 1200 WELS...
96,A1ZQJXZ5RKYJEL,https://www.amazon.com/sp?seller=A1ZQJXZ5RKYJEL,Nature Expert,Sepunsa 16478 BEACH BLVD # 286 WESTMINSTER CA ...
97,A2DF3HGBR01ATY,https://www.amazon.com/sp?seller=A2DF3HGBR01ATY,Garner Supply,Salem Group Corp 105 Maxess Rd ste 124 Melvill...
98,A1QZO01ZLATCA5,https://www.amazon.com/sp?seller=A1QZO01ZLATCA5,Whitestone D,WHITESTONE DEVELOPMENT LLC P.O. Box 32 Clarksb...


In [66]:
result.to_csv(datetime.strftime(datetime.now(), "%Y%m%d") + 'rfsellersaddress.csv', index=False)